# Pre-processing pipeline for spikeglx sessions, zebra finch / starling
- For every run in the session:
 - Load the recordings
 - Extract wav chan with micrhopohone and make a wav chan with the nidq syn signal
 - Get the sync events for the nidq sync channel
 
 - Do bout detection
 
In another notebook, bout detection is curated
- Left to decide where to:
    - Sort spikes
    - Sync the spikes/lfp/nidq
    - make and plot 'bout rasters'

In [1]:
%matplotlib inline
import os
import glob
import logging
import pickle
import numpy as np
import pandas as pd
from scipy.io import wavfile
from scipy import signal
import traceback
import warnings

from matplotlib import pyplot as plt
from importlib import reload

logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)


In [2]:
from ceciestunepipe.file import bcistructure as et
from ceciestunepipe.util import sglxutil as sglu
from ceciestunepipe.util import rigutil as ru
from ceciestunepipe.util import wavutil as wu
from ceciestunepipe.util import syncutil as su
from ceciestunepipe.util import stimutil as st
from ceciestunepipe.util.sound import boutsearch as bs
from ceciestunepipe.pipeline import searchbout as sb

from ceciestunepipe.util.spikeextractors import preprocess as pre
from ceciestunepipe.util.spikeextractors.extractors.spikeglxrecordingextractor import readSGLX as rsgl
from ceciestunepipe.util.spikeextractors.extractors.spikeglxrecordingextractor import spikeglxrecordingextractor as sglex

In [3]:
import spikeinterface as si
import spikeinterface.extractors as se
import spikeinterface.toolkit as st
import spikeinterface.sorters as ss
import spikeinterface.comparison as sc
import spikeinterface.widgets as sw
logger.info('all modules loaded')

2022-05-14 20:41:20,832 root         INFO     all modules loaded


## Session parameters and raw files

#### list all the sessions for this bird

In [4]:
bird = 's_b1238_22'
all_bird_sess = et.list_sessions(bird)
logger.info('all sessions for bird are {}'.format(all_bird_sess))

2022-05-14 20:41:21,359 root         INFO     all sessions for bird are ['2022-01-07', '2022-01-08', '2022-01-09', '2022-01-10', '2022-01-11', '2022-01-12', '2022-01-13', '2022-01-14', '2022-01-15', '2022-01-16', '2022-01-17', '2022-01-18', '2022-01-19', '2022-01-20', '2022-01-21', '2022-01-22', '2022-01-23', '2022-01-24', '2022-01-25', '2022-01-26', '2022-01-27', '2022-01-28', '2022-01-29', '2022-01-30', '2022-01-31', '2022-02-01', '2022-02-02', '2022-02-03', '2022-02-04', '2022-02-05', '2022-02-06', '2022-02-07', '2022-02-08', '2022-02-09', '2022-02-10', '2022-02-11', '2022-02-12', '2022-02-13', '2022-02-14', '2022-02-15', '2022-02-16', '2022-02-17', '2022-02-18', '2022-02-19', '2022-02-20', '2022-02-21', '2022-02-22', '2022-02-23', '2022-02-24', '2022-02-25', '2022-02-26', '2022-02-27', '2022-02-28', '2022-03-01', '2022-03-02', '2022-03-03', '2022-03-04', '2022-03-05', '2022-03-06', '2022-03-07', '2022-03-08', '2022-03-09', '2022-03-10', '2022-03-11', '2022-03-12', '2022-03-13', '20

### set up bird and sessions parameters
this will define:
- locations of files (for the bird)
- signals and channels to look for in the metadata of the files and in the rig.json parameter file: Note that this have to exist in all of the sessions that will be processed
- 'sess' is unimportant here, but it comes handy if there is need to debug usin a single session

In [5]:
reload(et)
# for one example session

sess_par = {'bird': 's_b1238_22',
           'sess': '2022-02-12',
           'probes': ['probe_0'], #probes of interest
           'mic_list': ['microphone_0'], #list of mics of interest, by signal name in rig.json
           'stim_list': ['wav_syn'], # list of adc chans with the stimulus
           'nidq_ttl_list': ['wav_ttl'], # list of TTL signals form the nidq digital inputs to extract (besides the 'sync')
           'sort': 2, #label for this sort instance
           }

exp_struct = et.get_exp_struct(sess_par['bird'], sess_par['sess'], sort=sess_par['sort'])

ksort_folder = exp_struct['folders']['ksort']
raw_folder = exp_struct['folders']['sglx']

list all the epochs in a session, to check that it is finding what it has to find

In [6]:
sess_epochs = et.list_sgl_epochs(sess_par)
sess_epochs

2022-05-14 20:41:26,829 ceciestunepipe.file.bcistructure INFO     {'folders': {'bird': '/mnt/sphere/speech_bci/raw_data/s_b1238_22', 'raw': '/mnt/sphere/speech_bci/raw_data/s_b1238_22/2022-02-12', 'sglx': '/mnt/sphere/speech_bci/raw_data/s_b1238_22/2022-02-12/sglx', 'kwik': '/scratch/earneodo/s_b1238_22/sglx/kwik/2022-02-12', 'processed': '/mnt/sphere/speech_bci/processed_data/s_b1238_22/2022-02-12/sglx', 'derived': '/mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-12/sglx', 'tmp': '/scratch/earneodo/tmp', 'msort': '/scratch/earneodo/s_b1238_22/sglx/msort/2022-02-12', 'ksort': '/scratch/earneodo/s_b1238_22/sglx/ksort/2022-02-12/2', 'sort': '/mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-12/sglx/2'}, 'files': {'par': '/scratch/earneodo/s_b1238_22/sglx/ksort/2022-02-12/2/params.json', 'set': '/mnt/sphere/speech_bci/raw_data/s_b1238_22/2022-02-12/sglx/settings.isf', 'rig': '/mnt/sphere/speech_bci/raw_data/s_b1238_22/2022-02-12/sglx/rig.json', 'kwd': '/scratch/earneodo/s_b123

['0716_g0', '1141_stim_g0', '1607_g0', '2212_g0']

#### define pre-processing steps for each epoch and for the session

###### for preprocessing one single epoch

In [7]:
reload(pre)
one_epoch_dict = pre.preprocess_run(sess_par, exp_struct, sess_epochs[2])

2022-05-14 20:41:35,089 ceciestunepipe.util.spikeextractors.preprocess INFO     PREPROCESSING sess 2022-02-12 | epoch 1607_g0
2022-05-14 20:41:35,090 ceciestunepipe.util.spikeextractors.preprocess INFO     getting extractors
2022-05-14 20:41:35,124 ceciestunepipe.util.spikeextractors.preprocess INFO     Got sglx recordings for keys ['nidq', 'lf_0', 'ap_0']
2022-05-14 20:41:35,125 ceciestunepipe.util.spikeextractors.preprocess INFO     Getting microphone channel(s) ['microphone_0']
2022-05-14 20:41:35,126 ceciestunepipe.util.wavutil INFO     sampling rate 40000
2022-05-14 20:41:35,126 ceciestunepipe.util.wavutil INFO     saving (1, 872783924)-shaped array as wav in /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-12/sglx/1607_g0/wav_mic.wav
2022-05-14 20:41:50,774 ceciestunepipe.util.wavutil INFO     saving (1, 872783924)-shaped array as npy in /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-12/sglx/1607_g0/wav_mic.npy
2022-05-14 20:42:07,672 ceciestunepipe.util.spikeextrac

2022-05-14 21:05:07,775 ceciestunepipe.util.spikeextractors.preprocess INFO     saving sync dict to /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-12/sglx/1607_g0/ap_0_sync_dict.pkl
2022-05-14 21:05:07,777 ceciestunepipe.util.spikeextractors.preprocess INFO     loading syn_dict from /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-12/sglx/1607_g0/ap_0_sync_dict.pkl
2022-05-14 21:05:07,782 ceciestunepipe.util.spikeextractors.preprocess INFO     getting syn patterns for nidq
2022-05-14 21:05:07,783 ceciestunepipe.util.spikeextractors.preprocess INFO     loading syn_dict from /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-12/sglx/1607_g0/nidq_sync_dict.pkl
2022-05-14 21:05:07,905 ceciestunepipe.util.spikeextractors.preprocess INFO     getting syn patterns for wav
2022-05-14 21:05:07,930 ceciestunepipe.util.spikeextractors.preprocess INFO     loading syn_dict from /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-12/sglx/1607_g0/wav_sync_dict.pkl
2022-05-14 21:05

In [8]:
### sequentially process all runs of the sessions
def preprocess_session(sess_par: dict):
    logger.info('pre-process all runs of sess ' + sess_par['sess'])
    # get exp struct
    sess_struct = et.get_exp_struct(sess_par['bird'], sess_par['sess'], sort=sess_par['sort'])
    # list the epochs
    sess_epochs = et.list_sgl_epochs(sess_par)
    logger.info('found epochs: {}'.format(sess_epochs))
    # preprocess all epochs
    epoch_dict_list = []
    for i_ep, epoch in enumerate(sess_epochs):
        try:
            exp_struct = et.sgl_struct(sess_par, epoch)
            one_epoch_dict = pre.preprocess_run(sess_par, exp_struct, epoch)
            epoch_dict_list.append(one_epoch_dict)
        except Exception as exc:
            warnings.warn('Error in epoch {}'.format(epoch), UserWarning)
            logger.info(traceback.format_exc)
            logger.info(exc)
            logger.info('Session {} epoch {} could not be preprocessed'.format(sess_par['sess'], epoch))
        
    return epoch_dict_list

#all_epoch_list = preprocess_session(sess_par)

## Process multiple sessions

In [9]:
sess_list = all_bird_sess
# fist implant, right hemisphere
#sess_list = ['2021-06-24', '2021-06-25', '2021-06-26', '2021-06-27', '2021-06-28', '2021-06-29', '2021-06-30']
sess_list = ['2022-01-17', '2022-01-18', '2022-01-19', '2022-01-20', # RH RA
             '2022-01-26', '2022-01-27','2022-01-28', '2022-01-29', '2022-01-30', #LH RA
            '2022-02-11', '2022-01-12', '2022-01-13'] #LH NCM

sess_list_ncm = ['2022-02-12']

In [10]:
sess_list = [sess_list[5]]

In [11]:
sess_list

['2022-01-27']

In [12]:
all_sess_dict = {}

for one_sess in sess_list[:]:
    sess_par['sess'] = one_sess
    preprocess_session(sess_par)

2022-05-14 21:05:08,512 root         INFO     pre-process all runs of sess 2022-01-27
2022-05-14 21:05:08,513 ceciestunepipe.file.bcistructure INFO     {'folders': {'bird': '/mnt/sphere/speech_bci/raw_data/s_b1238_22', 'raw': '/mnt/sphere/speech_bci/raw_data/s_b1238_22/2022-01-27', 'sglx': '/mnt/sphere/speech_bci/raw_data/s_b1238_22/2022-01-27/sglx', 'kwik': '/scratch/earneodo/s_b1238_22/sglx/kwik/2022-01-27', 'processed': '/mnt/sphere/speech_bci/processed_data/s_b1238_22/2022-01-27/sglx', 'derived': '/mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-01-27/sglx', 'tmp': '/scratch/earneodo/tmp', 'msort': '/scratch/earneodo/s_b1238_22/sglx/msort/2022-01-27', 'ksort': '/scratch/earneodo/s_b1238_22/sglx/ksort/2022-01-27/2', 'sort': '/mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-01-27/sglx/2'}, 'files': {'par': '/scratch/earneodo/s_b1238_22/sglx/ksort/2022-01-27/2/params.json', 'set': '/mnt/sphere/speech_bci/raw_data/s_b1238_22/2022-01-27/sglx/settings.isf', 'rig': '/mnt/sphere/spee

2022-05-14 21:18:50,938 ceciestunepipe.util.spikeextractors.preprocess INFO     loading syn_dict from /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-01-27/sglx/0605_g0/lf_0_sync_dict.pkl
2022-05-14 21:18:50,946 ceciestunepipe.util.spikeextractors.preprocess INFO     getting syn patterns for ap_0
2022-05-14 21:18:50,948 ceciestunepipe.util.spikeextractors.preprocess INFO     File /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-01-27/sglx/0605_g0/ap_0_sync_dict.pkl not found or forced computation, getting the events
2022-05-14 21:18:50,949 ceciestunepipe.util.spikeextractors.extractors.spikeglxrecordingextractor.spikeglxrecordingextractor INFO     getting ttl events, chan 0
2022-05-14 21:18:50,949 ceciestunepipe.util.spikeextractors.extractors.readSGLX INFO     Extracting digital on imec stream: there is only one channel, if any (SYN) and will be converted from an int16 channel 
2022-05-14 21:18:50,950 ceciestunepipe.util.spikeextractors.extractors.readSGLX INFO     allocated ar

2022-05-14 21:45:01,290 ceciestunepipe.util.spikeextractors.extractors.readSGLX INFO     allocated array for syn channel of size (49204354,)
2022-05-14 21:45:55,916 ceciestunepipe.util.spikeextractors.extractors.readSGLX INFO     Threshold for logical hi is 0
/mnt/cube/earneodo/repos/ceciestunepipe/ceciestunepipe/util/spikeextractors/extractors/spikeglxrecordingextractor/readSGLX.py:310: UserWarning: There were 1 errors in the recording
  warnings.warn('There were {} errors in the recording'.format(n_errors))
/mnt/cube/earneodo/repos/ceciestunepipe/ceciestunepipe/util/spikeextractors/extractors/spikeglxrecordingextractor/readSGLX.py:311: UserWarning: You probably want to check those
  warnings.warn('You probably want to check those')
2022-05-14 21:46:13,592 ceciestunepipe.util.spikeextractors.preprocess INFO     saving events array to /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-01-27/sglx/1154_g0/lf_0_sync_evt.npy
2022-05-14 21:46:13,634 ceciestunepipe.util.spikeextractors.prep

2022-05-14 22:06:46,255 ceciestunepipe.util.wavutil INFO     saving (1, 211903796)-shaped array as wav in /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-01-27/sglx/2005_g0/wav_mic.wav
2022-05-14 22:06:52,953 ceciestunepipe.util.wavutil INFO     saving (1, 211903796)-shaped array as npy in /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-01-27/sglx/2005_g0/wav_mic.npy
2022-05-14 22:06:57,427 ceciestunepipe.util.spikeextractors.preprocess INFO     Getting stimulus channel(s) ['wav_syn']
2022-05-14 22:06:57,430 ceciestunepipe.util.wavutil INFO     sampling rate 40000
2022-05-14 22:06:57,430 ceciestunepipe.util.wavutil INFO     saving (1, 211903796)-shaped array as wav in /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-01-27/sglx/2005_g0/wav_stim.wav
2022-05-14 22:07:01,616 ceciestunepipe.util.wavutil INFO     saving (1, 211903796)-shaped array as npy in /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-01-27/sglx/2005_g0/wav_stim.npy
2022-05-14 22:07:05,852 ceciestunepipe.ut

2022-05-14 22:16:23,384 ceciestunepipe.util.spikeextractors.preprocess INFO     loading syn_dict from /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-01-27/sglx/2005_g0/nidq_sync_dict.pkl
2022-05-14 22:16:23,390 ceciestunepipe.util.spikeextractors.preprocess INFO     getting syn patterns for wav
2022-05-14 22:16:23,391 ceciestunepipe.util.spikeextractors.preprocess INFO     loading syn_dict from /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-01-27/sglx/2005_g0/wav_sync_dict.pkl
2022-05-14 22:16:23,686 ceciestunepipe.util.spikeextractors.preprocess INFO     saved epoch preprocessing parameters to /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-01-27/sglx/2005_g0/preprocess_par.pickle
2022-05-14 22:16:23,733 ceciestunepipe.util.spikeextractors.preprocess INFO     PREPROCESSING sess 2022-01-27 | epoch 2135_nightstim_g0
2022-05-14 22:16:23,733 ceciestunepipe.util.spikeextractors.preprocess INFO     getting extractors
2022-05-14 22:16:23,917 ceciestunepipe.util.spikeextractors.

2022-05-14 22:50:32,903 ceciestunepipe.util.spikeextractors.preprocess INFO     saving sync dict to /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-01-27/sglx/2135_nightstim_g0/ap_0_sync_dict.pkl
2022-05-14 22:50:32,934 ceciestunepipe.util.spikeextractors.preprocess INFO     loading syn_dict from /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-01-27/sglx/2135_nightstim_g0/ap_0_sync_dict.pkl
2022-05-14 22:50:32,938 ceciestunepipe.util.spikeextractors.preprocess INFO     getting syn patterns for nidq
2022-05-14 22:50:32,939 ceciestunepipe.util.spikeextractors.preprocess INFO     File /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-01-27/sglx/2135_nightstim_g0/nidq_sync_dict.pkl not found or forced computation, getting the events
2022-05-14 22:50:32,940 ceciestunepipe.util.spikeextractors.extractors.spikeglxrecordingextractor.spikeglxrecordingextractor INFO     getting ttl events, chan 0
2022-05-14 22:56:42,551 ceciestunepipe.util.spikeextractors.preprocess INFO     saving eve

In [13]:
sess_par

{'bird': 's_b1238_22',
 'sess': '2022-01-27',
 'probes': ['probe_0'],
 'mic_list': ['microphone_0'],
 'stim_list': ['wav_syn'],
 'nidq_ttl_list': ['wav_ttl'],
 'sort': 2}

## search bouts for those sessions

In [14]:
from ceciestunepipe.util.sound import boutsearch as bs
from ceciestunepipe.util import wavutil as wu

from joblib import Parallel, delayed
import pickle
import sys

In [15]:
hparams = { # default parameters work well for starling
    # spectrogram
    'num_freq':1024, #1024# how many channels to use in a spectrogram #
    'preemphasis':0.97, 
    'frame_shift_ms':5, # step size for fft
    'frame_length_ms':10, #128 # frame length for fft FRAME SAMPLES < NUM_FREQ!!!
    'min_level_db':-55, # minimum threshold db for computing spe 
    'ref_level_db':110, # reference db for computing spec
    'sample_rate':None, # sample rate of your data
    
    # spectrograms
    'mel_filter': False, # should a mel filter be used?
    'num_mels':1024, # how many channels to use in the mel-spectrogram
    'fmin': 500, # low frequency cutoff for mel filter
    'fmax': 12000, # high frequency cutoff for mel filter
    
    # spectrogram inversion
    'max_iters':200,
    'griffin_lim_iters':20,
    'power':1.5,

    # Added for the searching
    'read_wav_fun': bs.read_wav_chan, # function for loading the wav_like_stream (has to returns fs, ndarray)
    'file_order_fun': bs.sess_file_id, # function for extracting the file id within the session
    'min_segment': 30, # Minimum length of supra_threshold to consider a 'syllable' (ms)
    'min_silence': 3000, # Minmum distance between groups of syllables to consider separate bouts (ms)
    'min_bout': 3000, # min bout duration (ms)
    'peak_thresh_rms': 0.55, # threshold (rms) for peak acceptance,
    'thresh_rms': 0.25, # threshold for detection of syllables
    'mean_syl_rms_thresh': 0.3, #threshold for acceptance of mean rms across the syllable (relative to rms of the file)
    'max_bout': 180000, #exclude bouts too long
    'l_p_r_thresh': 100, # threshold for n of len_ms/peaks (typycally about 2-3 syllable spans
    
    'waveform_edges': 1000, #get number of ms before and after the edges of the bout for the waveform sample
    
    'bout_auto_file': 'bout_auto.pickle', # extension for saving the auto found files
    'bout_curated_file': 'bout_checked.pickle', #extension for manually curated files (coming soon)
    }

In [22]:
reload(sb)

<module 'ceciestunepipe.pipeline.searchbout' from '/mnt/cube/earneodo/repos/ceciestunepipe/ceciestunepipe/pipeline/searchbout.py'>

In [23]:
all_sessions = sess_list[:]
#all_sessions = ['2021-06-15']

for sess in all_sessions:
    sess_par['sess'] = sess
    sess_bout_pd = sb.get_all_day_bouts(sess_par, hparams, n_jobs=1, ephys_software='sglx', save=False)
    sb.save_auto_bouts(sess_bout_pd, sess_par, hparams, software='sglx')
    #sess_bouts_folder = os.path.join(exp_struct['folders']['derived'], 'bouts')
    #bouts_to_wavs(sess_bout_pd, sess_par, hparams, sess_bouts_folder)

2022-05-15 13:22:22,411 ceciestunepipe.pipeline.searchbout INFO     Will search for bouts through all session s_b1238_22, 2022-01-27
2022-05-15 13:22:23,133 ceciestunepipe.pipeline.searchbout INFO     getting wav files from/mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-01-27/sglx
2022-05-15 13:22:23,133 ceciestunepipe.pipeline.searchbout INFO     Found 5 files
2022-05-15 13:22:23,135 ceciestunepipe.util.sound.boutsearch INFO     Getting bouts for long file /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-01-27/sglx/0605_g0/wav_mic.wav
2022-05-15 13:22:23,137 ceciestunepipe.util.sound.boutsearch INFO     splitting file into 10 chunks


  0%|          | 0/10 [00:00<?, ?it/s]

2022-05-15 14:05:37,077 ceciestunepipe.util.sound.boutsearch INFO     Getting bouts for long file /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-01-27/sglx/1154_g0/wav_mic.wav
2022-05-15 14:05:37,094 ceciestunepipe.util.sound.boutsearch INFO     splitting file into 9 chunks


  0%|          | 0/9 [00:00<?, ?it/s]

2022-05-15 14:32:39,334 ceciestunepipe.util.sound.boutsearch INFO     Getting bouts for long file /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-01-27/sglx/1722_g0/wav_mic.wav
2022-05-15 14:32:39,339 ceciestunepipe.util.sound.boutsearch INFO     splitting file into 5 chunks


...

2022-05-15 14:36:34,814 ceciestunepipe.util.sound.boutsearch INFO     Getting bouts for long file /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-01-27/sglx/2135_nightstim_g0/wav_mic.wav
2022-05-15 14:36:34,817 ceciestunepipe.util.sound.boutsearch INFO     splitting file into 12 chunks


  0%|          | 0/12 [00:00<?, ?it/s]

2022-05-15 14:42:57,260 ceciestunepipe.pipeline.searchbout INFO     getting spectrograms
2022-05-15 14:46:15,377 ceciestunepipe.pipeline.searchbout INFO     saving bouts pandas to /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-01-27/bouts_sglx/bout_auto.pickle


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/earneodo/anaconda3/envs/unepipe/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3369, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-23-67427f5c28af>", line 7, in <cell line: 4>
    sb.save_auto_bouts(sess_bout_pd, sess_par, hparams, software='sglx')
  File "/mnt/cube/earneodo/repos/ceciestunepipe/ceciestunepipe/pipeline/searchbout.py", line 136, in save_auto_bouts
    fu.chmod(sess_bouts_path, 0o777)
  File "/mnt/cube/earneodo/repos/ceciestunepipe/ceciestunepipe/util/fileutil.py", line 22, in chmod
    os.chmod(file_path, mode)
PermissionError: [Errno 1] Operation not permitted: '/mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-01-27/bouts_sglx/bout_auto.pickle'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/earneodo/anaconda3/envs/unepipe/lib/python3.8/site-packages/IPython/core/interactiveshell.py", 

In [26]:
sb.save_auto_bouts(sess_bout_pd, sess_par, hparams, software='sglx')

2022-05-16 07:13:12,155 ceciestunepipe.pipeline.searchbout INFO     saving bouts pandas to /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-01-27/bouts_sglx/bout_auto.pickle
2022-05-16 07:13:28,384 ceciestunepipe.pipeline.searchbout INFO     saving bout detect parameters dict to /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-01-27/bouts_sglx/bout_search_params.pickle
2022-05-16 07:13:28,385 ceciestunepipe.util.sound.boutsearch INFO     saving bout detect parameters dict to /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-01-27/bouts_sglx/bout_search_params.pickle


PermissionError: [Errno 1] Operation not permitted: '/mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-01-27/bouts_sglx/bout_search_params.pickle'

In [24]:
sess_bout_pd.head(2)

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,syl_in,n_syl,peaks_p,n_peaks,l_p_ratio,waveform,confusing,valid_waveform,valid,spectrogram
0,22120,57770,884800,2310800,"[9.028354520968774, 11.430471911611551, 13.209...",29.550182,273.312366,False,/mnt/sphere/speech_bci/derived_data/s_b1238_22...,35650,"[[22120, 23205], [25700, 25745], [27200, 27250...",89,"[5, 13, 20, 31, 40, 52, 67, 74, 85, 97, 104, 1...",381,93.569554,"[-47, -54, -43, -49, -65, -50, -57, -56, -55, ...",True,True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,66030,96410,2641200,3856400,"[8.904722885893623, 9.394329271872063, 15.6020...",29.550182,277.671870,False,/mnt/sphere/speech_bci/derived_data/s_b1238_22...,30380,"[[66030, 66965], [69505, 69605], [71305, 71350...",72,"[18, 26, 37, 44, 52, 60, 71, 81, 104, 117, 129...",344,88.313953,"[19, 31, 26, 32, 44, 48, 28, 17, 13, 12, 9, 6,...",True,True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [18]:
sb.save_auto_bouts(sess_bout_pd, sess_par, hparams, software='sglx')

2022-04-12 09:17:02,653 ceciestunepipe.pipeline.searchbout INFO     saving bouts pandas to /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-12/bouts_sglx/bout_auto.pickle
2022-04-12 09:17:31,269 ceciestunepipe.pipeline.searchbout INFO     saving bout detect parameters dict to /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-12/bouts_sglx/bout_search_params.pickle
2022-04-12 09:17:31,270 ceciestunepipe.util.sound.boutsearch INFO     saving bout detect parameters dict to /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-12/bouts_sglx/bout_search_params.pickle


# debug

## debug search_bout

In [21]:
## look for a single file
sess = sess_list[0]

exp_struct = et.get_exp_struct(sess_par['bird'], sess, ephys_software='sglx')
source_folder = exp_struct['folders']['derived']
wav_path_list = et.get_sgl_files_epochs(source_folder, file_filter='*wav_mic.wav')
wav_path_list.sort()
logger.info('Found {} files'.format(len(wav_path_list)))
print(wav_path_list)

2021-09-22 15:13:39,371 root         INFO     Found 4 files


['/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-14/sglx/0712_g0/wav_mic.wav', '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-14/sglx/1255_g0/wav_mic.wav', '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-14/sglx/1740_g0/wav_mic.wav', '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-14/sglx/2118_g0/wav_mic.wav']


In [22]:
one_file = wav_path_list[0]

In [ ]:
reload(bs)
epoch_bout_pd, epoch_wav = bs.get_bouts_in_long_file(wav_path_list[0], hparams)

2021-09-22 15:13:45,924 ceciestunepipe.util.sound.boutsearch INFO     Getting bouts for long file /mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-14/sglx/0712_g0/wav_mic.wav


tu vieja file /mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-14/sglx/0712_g0/wav_mic.wav


2021-09-22 15:13:45,962 ceciestunepipe.util.sound.boutsearch INFO     splitting file into 5 chunks


  0%|          | 0/5 [00:00<?, ?it/s]